## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint
7. After this lab is completed, delete the endpoint to avoid unnecessary charges

In [1]:
# Install SageMaker 2.x version.
!pip install --upgrade sagemaker

Using legacy 'setup.py install' for sagemaker, since package 'wheel' is not installed.
Using legacy 'setup.py install' for protobuf3-to-dict, since package 'wheel' is not installed.
    Running setup.py install for protobuf3-to-dict: started
    Running setup.py install for protobuf3-to-dict: finished with status 'done'
    Running setup.py install for sagemaker: started
    Running setup.py install for sagemaker: finished with status 'done'


In [2]:
import boto3
import sagemaker
import math
import dateutil

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [3]:
sagemaker.__version__

'2.27.0'

In [4]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='ml-usr-predict',region_name='us-east-1')

ProfileNotFound: The config profile (ml-usr-predict) could not be found

In [ ]:
sess = sagemaker.Session(boto_session=boto_session)

In [ ]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
endpoint_name = 'xgboost-bikerental-v1'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [ ]:
# We are sending data for inference in CSV format
predictor.serializer = CSVSerializer()

In [ ]:
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [ ]:
#predictor.predict(sample_one)

In [ ]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [ ]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

In [ ]:
# Let's invoke prediction now
predictor.predict(transform_data(sample_one))

In [ ]:
# Actual Count is 562...but predicted is 6.3.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
result = predictor.predict(transform_data(sample_one))
result = result.decode("utf-8")
print ('Predicted Count', math.expm1(float(result)))

In [ ]:
# how to send multiple samples
result = predictor.predict([transform_data(sample_one), transform_data(sample_two)])

In [ ]:
result.decode("utf-8")

In [ ]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = predictor.predict(transformed_data[i : i + batch_size])
        
        result = result.decode("utf-8")
        result = result.split(',')
        
        predictions += [math.expm1(float(r)) for r in result]
                
    return predictions

In [ ]:
run_predictions([sample_one,sample_two,sample_three],10)

In [ ]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [ ]:
len(data)

In [ ]:
%%time
predictions = run_predictions(data,500)

In [ ]:
len(predictions),len(data)

In [ ]:
#print(predictions)

In [ ]:
# Don't forget to delete the endpoint
# From SageMaker Console, Select "Endpoints" under Inference and Delete the Endpoint